In [1]:
import credentials
import settings

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

class MyStreamerListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return False
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)
        sentiment = TextBlob(status.text)
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count = status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude= status.coordinates['coordinates']
            latitude = status.coordinates['coordinates']
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        print(status.text)
        print("Long: {}, Lat: {}". format(longitude, latitude))
        
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location,user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql,val)
            mydb.commit()
            mycursor.close()


In [3]:
def clean_tweet(self, tweet): 

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
                                
def deEmojify(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [4]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="gaurav",
    database="TwitterDB",
    charset = 'utf8'
)

if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()


In [5]:
auth  = tweepy.OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [6]:
myStreamerListener = MyStreamerListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamerListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
mydb.close()

 for 50,000 Sats on Lightning!

2 Winners in 12 Hours!⚡️

Support Bitsahara: partners@bit…
Long: None, Lat: None
RT @relayxio: Launching Trademail: Convert BSV to any crypto using any BitCoin wallet via Paymail.

https://t.co/Wr3fYTvw6V
Long: None, Lat: None
RT @nebraskangooner: #bitcoin

We've seen this time and time again recently... https://t.co/mrUfiJbvO4
Long: None, Lat: None
@MWichers @rpthis2 @satoshi_babe If transitional markets have bottomed AND that bitcoin is not correlated with said… https://t.co/e1BvDZdX5n
Long: None, Lat: None
Trading #Bitcoin feel like trading a low cap penny stock, with a couple more zero. Lol

A Comparison of #Volatility… https://t.co/KvB8VrNw9u
Long: None, Lat: None
RT @Crypto3032: 1st giveaway 50 $ETH by @AlaminAhsan18
cmon, try your lucky 🧐💎

@cctip_io airdrop 10000000 CRU 250
Long: None, Lat: None
@piratebeachbum Last month my coworker asked me if they should buy BTC. I said, invest what you can afford. He bought 1 full Bitcoin at $10k
Long: None, 

MySQLInterfaceError: Python type list cannot be converted